# 📊 01_EDA — Exploratory Data Analysis
## CDISCPILOT01: 알츠하이머 임상시험 환자 탈락 예측 프로젝트

### 목적
- 각 SDTM 도메인(DM, AE, DS, SV, VS, CM, MH)의 구조와 내용 파악
- 타겟 변수(탈락 여부) 정의 및 분포 확인
- 탈락에 영향을 미칠 수 있는 주요 변수 탐색
- 도메인 간 관계 및 패턴 발견

### 데이터 출처
- **Study**: CDISC Pilot 01 (CDISCPILOT01) — Eli Lilly의 Xanomeline 알츠하이머 임상시험
- **Source**: [PhUSE Scripts Repository](https://github.com/phuse-org/phuse-scripts/tree/master/data/sdtm/cdiscpilot01)
- **Standard**: SDTM Version 2

## 1. 환경 설정 및 데이터 로드

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import warnings
warnings.filterwarnings('ignore')

# Style settings
plt.style.use('seaborn-v0_8-whitegrid')
sns.set_palette('Set2')
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 100)

In [2]:
# Load all SDTM domains
dm = pd.read_sas('../data/raw/dm.xpt', format='xport', encoding='utf-8')
ae = pd.read_sas('../data/raw/ae.xpt', format='xport', encoding='utf-8')
ds = pd.read_sas('../data/raw/ds.xpt', format='xport', encoding='utf-8')
sv = pd.read_sas('../data/raw/sv.xpt', format='xport', encoding='utf-8')
vs = pd.read_sas('../data/raw/vs.xpt', format='xport', encoding='utf-8')
cm = pd.read_sas('../data/raw/cm.xpt', format='xport', encoding='utf-8')
mh = pd.read_sas('../data/raw/mh.xpt', format='xport', encoding='utf-8')

# Domain overview
domains = {'DM': dm, 'AE': ae, 'DS': ds, 'SV': sv, 'VS': vs, 'CM': cm, 'MH': mh}
for name, df in domains.items():
    print(f"{name}: {df.shape[0]:,} rows × {df.shape[1]} cols | Subjects: {df['USUBJID'].nunique()}")

DM: 306 rows × 25 cols | Subjects: 306
AE: 1,191 rows × 35 cols | Subjects: 225
DS: 596 rows × 13 cols | Subjects: 306
SV: 3,559 rows × 8 cols | Subjects: 306
VS: 29,643 rows × 24 cols | Subjects: 254
CM: 7,510 rows × 21 cols | Subjects: 229
MH: 1,818 rows × 19 cols | Subjects: 254


## 2. DM (Demographics) — 인구통계 분석

DM은 환자의 기본 정보를 담고 있는 핵심 도메인입니다.  
Screen Failure(52명)를 제외한 실제 등록 환자 254명을 분석 대상으로 합니다.

In [ ]:
# Screen Failure 제외
dm_enrolled = dm[dm['ARM'] != 'Screen Failure'].copy()
print(f"등록 환자 수: {len(dm_enrolled)} (Screen Failure {len(dm) - len(dm_enrolled)}명 제외)")
print(f"
투약군 분포:")
print(dm_enrolled['ARM'].value_counts().to_frame())

In [ ]:
# ARM vs ACTARM 차이 확인
# ARM: 배정된 군, ACTARM: 실제 투약된 군
print("ARM (배정군) vs ACTARM (실제 투약군) 비교:")
print(pd.crosstab(dm_enrolled['ARM'], dm_enrolled['ACTARM']))

In [ ]:
# Demographics 요약
print("=" * 50)
print("📌 인구통계 요약")
print("=" * 50)

print(f"
나이: 평균 {dm_enrolled['AGE'].mean():.1f}세 (범위: {dm_enrolled['AGE'].min():.0f}-{dm_enrolled['AGE'].max():.0f})")
print(f"
성별:")
print(dm_enrolled['SEX'].value_counts().to_frame())
print(f"
인종:")
print(dm_enrolled['RACE'].value_counts().to_frame())
print(f"
사이트 수: {dm_enrolled['SITEID'].nunique()}")
print(f"국가: {dm_enrolled['COUNTRY'].unique()}")

In [ ]:
# DM 결측치 확인
print("DM 결측치:")
missing = dm_enrolled.isnull().sum()
missing = missing[missing > 0]
for col, cnt in missing.items():
    print(f"  {col}: {cnt} ({cnt/len(dm_enrolled)*100:.1f}%)")

## 3. DS (Disposition) — 타겟 변수 정의

DS 도메인에서 **DISPOSITION EVENT** 카테고리의 DSDECOD를 기준으로 타겟 변수를 생성합니다.
-  → 0 (시험 완료)
- 그 외 → 1 (탈락/중도 중단)

In [ ]:
# 타겟 변수 생성
enrolled_ids = set(dm_enrolled['USUBJID'])
disp = ds[ds['DSCAT'] == 'DISPOSITION EVENT'].copy()
disp = disp[disp['USUBJID'].isin(enrolled_ids)]
disp['DROPOUT'] = (disp['DSDECOD'] != 'COMPLETED').astype(int)

print(f"시험 완료: {(disp['DROPOUT']==0).sum()}명 ({(disp['DROPOUT']==0).mean()*100:.1f}%)")
print(f"탈    락: {(disp['DROPOUT']==1).sum()}명 ({(disp['DROPOUT']==1).mean()*100:.1f}%)")
print(f"
탈락 사유 상세:")
print(disp[disp['DROPOUT']==1]['DSDECOD'].value_counts().to_frame())

In [ ]:
# 투약군별 탈락률
fig, axes = plt.subplots(1, 3, figsize=(16, 5))

# (1) Completed vs Dropout
completed = (disp['DROPOUT']==0).sum()
dropout = (disp['DROPOUT']==1).sum()
bars = axes[0].bar(['Completed', 'Dropout'], [completed, dropout], 
                     color=['#2ecc71', '#e74c3c'], edgecolor='white', width=0.6)
for bar in bars:
    axes[0].text(bar.get_x()+bar.get_width()/2, bar.get_height()+2, 
                 f'{int(bar.get_height())}', ha='center', fontweight='bold', fontsize=13)
axes[0].set_title('Completed vs Dropout', fontweight='bold')
axes[0].set_ylabel('Number of Subjects')

# (2) Dropout Reasons
reasons = disp[disp['DROPOUT']==1]['DSDECOD'].value_counts()
axes[1].barh(range(len(reasons)), reasons.values, color=sns.color_palette('Reds_r', len(reasons)))
axes[1].set_yticks(range(len(reasons)))
axes[1].set_yticklabels(reasons.index, fontsize=9)
for i, v in enumerate(reasons.values):
    axes[1].text(v+0.5, i, str(v), va='center', fontweight='bold')
axes[1].set_title('Dropout Reasons', fontweight='bold')
axes[1].invert_yaxis()

# (3) Dropout Rate by ARM
merged = disp.merge(dm_enrolled[['USUBJID', 'ARM']], on='USUBJID')
arm_dropout = merged.groupby('ARM')['DROPOUT'].agg(['sum', 'count'])
arm_dropout['rate'] = arm_dropout['sum'] / arm_dropout['count'] * 100
arm_dropout = arm_dropout.loc[['Placebo', 'Xanomeline Low Dose', 'Xanomeline High Dose']]
colors = ['#3498db', '#f39c12', '#e74c3c']
bars = axes[2].bar(['Placebo', 'Low Dose', 'High Dose'], arm_dropout['rate'], color=colors, width=0.6)
for bar, (_, row) in zip(bars, arm_dropout.iterrows()):
    axes[2].text(bar.get_x()+bar.get_width()/2, bar.get_height()+1,
                 f'{row["rate"]:.1f}%
({int(row["sum"])}/{int(row["count"])})',
                 ha='center', fontweight='bold')
axes[2].set_title('Dropout Rate by Treatment Arm', fontweight='bold')
axes[2].set_ylabel('Dropout Rate (%)')
axes[2].set_ylim(0, 100)

plt.tight_layout()
plt.savefig('../outputs/figures/dropout_overview.png', dpi=150, bbox_inches='tight')
plt.show()

### 💡 핵심 발견
- **전체 탈락률 56.7%** — 절반 이상이 시험을 완료하지 못함
- **탈락 사유 1위: Adverse Event (92건, 63.9%)** — 부작용으로 인한 중단이 압도적
- **Xanomeline 투여군의 탈락률이 약 70%**로, Placebo(32.6%)보다 2배 이상 높음
- 이는 xanomeline의 피부 부작용(패치 제형)과 일치하는 패턴

## 4. AE (Adverse Events) — 이상반응 분석

In [ ]:
# 등록 환자의 AE만 필터
ae_enrolled = ae[ae['USUBJID'].isin(enrolled_ids)].copy()
print(f"총 이상반응 건수: {len(ae_enrolled)}")
print(f"AE 경험 환자: {ae_enrolled['USUBJID'].nunique()}/{len(dm_enrolled)} ({ae_enrolled['USUBJID'].nunique()/len(dm_enrolled)*100:.1f}%)")

print(f"
심각도 (AESEV):")
print(ae_enrolled['AESEV'].value_counts().to_frame())
print(f"
SAE (AESER):")
print(ae_enrolled['AESER'].value_counts().to_frame())
print(f"
약물 관련성 (AEREL):")
print(ae_enrolled['AEREL'].value_counts().to_frame())

In [ ]:
# Top 15 이상반응
print("Top 15 이상반응 (AEDECOD):")
print(ae_enrolled['AEDECOD'].value_counts().head(15).to_frame())
print("
→ 상위 이상반응이 대부분 피부/적용부위 관련 (패치 제형 특성)")

In [ ]:
# AE와 Dropout의 관계
ae_count = ae_enrolled.groupby('USUBJID').size().reset_index(name='ae_count')
all_subj = pd.DataFrame({'USUBJID': list(enrolled_ids)})
ae_count = all_subj.merge(ae_count, on='USUBJID', how='left').fillna(0)
ae_count = ae_count.merge(disp[['USUBJID', 'DROPOUT']], on='USUBJID')
ae_count['Dropout_Label'] = ae_count['DROPOUT'].map({0: 'Completed', 1: 'Dropout'})

fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# AE count by dropout
for label, color in [('Completed', '#2ecc71'), ('Dropout', '#e74c3c')]:
    data = ae_count[ae_count['Dropout_Label']==label]['ae_count']
    axes[0].hist(data, bins=20, alpha=0.6, color=color, 
                 label=f'{label} (mean={data.mean():.1f})', edgecolor='white')
axes[0].set_title('AE Count Distribution by Dropout Status', fontweight='bold')
axes[0].set_xlabel('Number of AEs per Subject')
axes[0].legend()

# AE Severity by ARM
ae_arm = ae_enrolled.merge(dm_enrolled[['USUBJID', 'ARM']], on='USUBJID')
ct = pd.crosstab(ae_arm['ARM'], ae_arm['AESEV'], normalize='index') * 100
ct = ct.loc[['Placebo', 'Xanomeline Low Dose', 'Xanomeline High Dose']]
ct.index = ['Placebo', 'Low Dose', 'High Dose']
ct[['MILD', 'MODERATE', 'SEVERE']].plot(kind='bar', ax=axes[1],
    color=['#2ecc71', '#f39c12', '#e74c3c'], edgecolor='white', rot=0)
axes[1].set_title('AE Severity by Treatment Arm (%)', fontweight='bold')
axes[1].set_ylabel('Percentage')

plt.tight_layout()
plt.savefig('../outputs/figures/ae_analysis.png', dpi=150, bbox_inches='tight')
plt.show()

## 5. SV (Subject Visits) — 방문 기록 분석

In [ ]:
# 등록 환자 방문 기록
sv_enrolled = sv[sv['USUBJID'].isin(enrolled_ids)].copy()
print(f"총 방문 기록: {len(sv_enrolled)}")
print(f"
방문 종류:")
print(sv_enrolled['VISIT'].value_counts().sort_index())

In [ ]:
# 정규 방문만 필터 (Unscheduled, Retrieval, AE Follow-up 제외)
sv_scheduled = sv_enrolled[~sv_enrolled['VISIT'].str.contains('UNSCHEDULED|RETRIEVAL|AE FOLLOW', na=False)]
visit_count = sv_scheduled.groupby('USUBJID').size().reset_index(name='visit_count')
visit_count = visit_count.merge(disp[['USUBJID', 'DROPOUT']], on='USUBJID')
visit_count['Dropout_Label'] = visit_count['DROPOUT'].map({0: 'Completed', 1: 'Dropout'})

fig, ax = plt.subplots(figsize=(8, 5))
sns.boxplot(data=visit_count, x='Dropout_Label', y='visit_count',
            palette={'Completed': '#2ecc71', 'Dropout': '#e74c3c'}, ax=ax)
ax.set_title('Scheduled Visit Count by Dropout Status', fontweight='bold')
ax.set_xlabel('')
ax.set_ylabel('Number of Scheduled Visits')

# Add mean annotations
for i, label in enumerate(['Completed', 'Dropout']):
    mean = visit_count[visit_count['Dropout_Label']==label]['visit_count'].mean()
    ax.text(i, mean, f'  mean={mean:.1f}', va='center', fontweight='bold', color='black')

plt.tight_layout()
plt.savefig('../outputs/figures/visit_analysis.png', dpi=150, bbox_inches='tight')
plt.show()

## 6. CM & MH — 병용약물 및 병력 분석

In [ ]:
# CM 분석
cm_enrolled = cm[cm['USUBJID'].isin(enrolled_ids)]
cm_count = cm_enrolled.groupby('USUBJID').size().reset_index(name='cm_count')
cm_count = all_subj.merge(cm_count, on='USUBJID', how='left').fillna(0)
cm_count = cm_count.merge(disp[['USUBJID', 'DROPOUT']], on='USUBJID')

# MH 분석
mh_enrolled = mh[mh['USUBJID'].isin(enrolled_ids)]
mh_count = mh_enrolled.groupby('USUBJID').size().reset_index(name='mh_count')
mh_count = all_subj.merge(mh_count, on='USUBJID', how='left').fillna(0)
mh_count = mh_count.merge(disp[['USUBJID', 'DROPOUT']], on='USUBJID')

fig, axes = plt.subplots(1, 2, figsize=(14, 5))

for i, (df, col, title) in enumerate([
    (cm_count, 'cm_count', 'Concomitant Medications Count'),
    (mh_count, 'mh_count', 'Medical History Count')
]):
    df['Dropout_Label'] = df['DROPOUT'].map({0: 'Completed', 1: 'Dropout'})
    sns.boxplot(data=df, x='Dropout_Label', y=col,
                palette={'Completed': '#2ecc71', 'Dropout': '#e74c3c'}, ax=axes[i])
    axes[i].set_title(f'{title} by Dropout Status', fontweight='bold')
    axes[i].set_xlabel('')

plt.tight_layout()
plt.savefig('../outputs/figures/cm_mh_analysis.png', dpi=150, bbox_inches='tight')
plt.show()

## 7. VS (Vital Signs) — 활력징후 분석

In [ ]:
# VS 기본 정보
vs_enrolled = vs[vs['USUBJID'].isin(enrolled_ids)]
print("측정 항목:")
print(vs_enrolled.groupby('VSTESTCD')['VSSTRESN'].describe().round(1))

In [ ]:
# Baseline 활력징후 by Dropout
vs_bl = vs_enrolled[vs_enrolled['VSBLFL'] == 'Y'].copy()
vs_bl = vs_bl.merge(disp[['USUBJID', 'DROPOUT']], on='USUBJID')
vs_bl['Dropout_Label'] = vs_bl['DROPOUT'].map({0: 'Completed', 1: 'Dropout'})

fig, axes = plt.subplots(2, 3, figsize=(16, 10))
fig.suptitle('Baseline Vital Signs by Dropout Status', fontsize=14, fontweight='bold')

for i, testcd in enumerate(['SYSBP', 'DIABP', 'PULSE', 'TEMP', 'WEIGHT', 'HEIGHT']):
    ax = axes[i//3][i%3]
    data = vs_bl[vs_bl['VSTESTCD'] == testcd]
    if len(data) > 0:
        sns.boxplot(data=data, x='Dropout_Label', y='VSSTRESN',
                    palette={'Completed': '#2ecc71', 'Dropout': '#e74c3c'}, ax=ax)
        ax.set_title(f'{testcd}', fontweight='bold')
        ax.set_xlabel('')

plt.tight_layout()
plt.savefig('../outputs/figures/vs_baseline.png', dpi=150, bbox_inches='tight')
plt.show()

## 8. EDA 종합 요약

### 📊 데이터 규모
| 항목 | 값 |
|------|-----|
| 총 등록 환자 | 254명 (Screen Failure 52명 제외) |
| 투약군 | Placebo(86), Low Dose(84), High Dose(84) |
| 시험 완료 | 110명 (43.3%) |
| 탈락 | 144명 (56.7%) |

### 🔍 핵심 발견

**1. 탈락률이 매우 높다 (56.7%)**
- 탈락 사유 1위: Adverse Event (63.9%)
- Xanomeline 투여군(~70%)이 Placebo(32.6%)보다 탈락률이 2배 이상 높음

**2. 이상반응(AE) 패턴**
- 전체 환자의 88.6%가 1건 이상의 AE 경험
- 상위 AE: 가려움, 적용부위 반응, 홍반, 발진 → 패치 제형 특성
- 투약군일수록 MODERATE/SEVERE 비율 증가
- 투약군의 약물 관련 AE (PROBABLE) 비율이 Placebo보다 높음

**3. 방문 기록**
- 탈락 환자는 당연히 방문 횟수가 적음 → 초기 방문 패턴이 예측 피처가 될 수 있음

**4. 인구통계**
- 평균 연령 75세, 여성 비율이 약간 높음
- 나이, 성별에 따른 탈락률 차이는 크지 않음 (추후 통계 검정 필요)

### ➡️ 다음 단계 (02_preprocessing.ipynb)
1. SDTM 도메인 간 USUBJID 기준 병합
2. 도메인 지식 기반 피처 엔지니어링
3. 결측치 처리 및 인코딩
4. 최종 분석 데이터셋 생성